<a href="https://colab.research.google.com/github/AstroBesat-SoftW/AstroBesat-SoftW/blob/main/7_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import pickle
import joblib
import xgboost as xgb
import lightgbm as lgb
import os
import tensorflow as tf
import random

# Keras / TensorFlow
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import Callback

# Scikit-Learn Modelleri
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report

# --- AYARLAR ---
MAX_EPOCH_LIMIT = 40 # CNN epoch limiti
BATCH_SIZE = 32

# Rastgelelikleri sabitle
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

print(">> SİSTEM BAŞLATILIYOR (FULL MODEL MODU)...")

# --- KULLANICIYA SORU ---
print("\n" + "="*50)
print("🤔 EĞİTİM STRATEJİSİ")
muallak_sayisi = input(">> Kaç adet 'MUALLAK' (Arada kalmış) veri üreteyim? (Örn: 200): ")
try:
    muallak_sayisi = int(muallak_sayisi)
except:
    muallak_sayisi = 0
    print(">> 0 kabul edildi.")
print("="*50 + "\n")

# --- 1. VERİ ÜRETİM FONKSİYONLARI (İYİLEŞTİRİLMİŞ) ---

def generate_sequence(length, vocab, pattern=None):
    seq = list(np.random.choice(list(vocab), size=length))
    if pattern:
        start = np.random.randint(0, length - len(pattern) + 1)
        for i, char in enumerate(pattern):
            seq[start + i] = char
    return "".join(seq)

def veri_uret_standart(n_patojenik, n_benign):
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    total = n_patojenik + n_benign

    for i in range(total):
        label = 1 if i < n_patojenik else 0

        # Sinyal Güçlendirme (Motifler)
        if label == 1:
            dna = generate_sequence(11, "ACGT", pattern="CGT")
            prot = generate_sequence(11, amino_acids, pattern="WW")
            risk = np.random.beta(8, 2)
            cons = np.random.uniform(7, 10)
        else:
            dna = generate_sequence(11, "ACGT", pattern="AAA")
            prot = generate_sequence(11, amino_acids, pattern="LL")
            risk = np.random.beta(2, 8)
            cons = np.random.uniform(0, 3)

        maf = np.random.exponential(0.05)
        hydro = np.random.uniform(-5, 5)
        polar = np.random.uniform(-3, 3)
        weight = np.random.uniform(10, 80) if label == 1 else np.random.uniform(-20, 40)

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)
    return dna_seqs, prot_seqs, bio_feats, labels

def veri_uret_muallak(adet):
    print(f"   ⚠️ {adet} adet 'Muallak/Zor' veri üretiliyor...")
    dna_seqs, prot_seqs, bio_feats, labels = [], [], [], []
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"

    for _ in range(adet):
        risk = np.random.uniform(0.40, 0.65)
        cons = np.random.uniform(4, 6)
        maf = np.random.uniform(0.01, 0.1)
        hydro = np.random.uniform(-2, 2)
        polar = np.random.uniform(-1, 1)
        weight = np.random.uniform(-10, 30)

        score = (risk * 0.6) + (cons/10 * 0.4)
        label = 1 if score > 0.52 else 0

        # Ters sinyal (Modeli zorlamak için)
        if np.random.random() > 0.8:
            dna_pattern = "AAA" if label == 1 else "CGT"
        else:
            dna_pattern = "CGT" if label == 1 else "AAA"

        dna = generate_sequence(11, "ACGT", pattern=dna_pattern)
        prot = generate_sequence(11, amino_acids)

        dna_seqs.append(dna)
        prot_seqs.append(prot)
        bio_feats.append([risk, maf, cons, hydro, polar, weight])
        labels.append(label)

    return dna_seqs, prot_seqs, bio_feats, labels

# --- 2. VERİ HAZIRLAMA ---
def veri_hazirla():
    all_dna, all_prot, all_feats, all_labels = [], [], [], []

    # Eğitim Verisi
    train_dagilim = {"Genel": (1800, 1800), "Kanser": (250, 250), "Nadir": (100, 100)}
    print(">> Veriler Harmanlanıyor...")
    for _, (p, b) in train_dagilim.items():
        d, pr, f, l = veri_uret_standart(p, b)
        all_dna.extend(d); all_prot.extend(pr); all_feats.extend(f); all_labels.extend(l)

    if muallak_sayisi > 0:
        d_m, p_m, f_m, l_m = veri_uret_muallak(muallak_sayisi)
        all_dna.extend(d_m); all_prot.extend(p_m); all_feats.extend(f_m); all_labels.extend(l_m)

    combined = list(zip(all_dna, all_prot, all_feats, all_labels))
    np.random.shuffle(combined)
    dna, prot, feats, labels = zip(*combined)
    return list(dna), list(prot), np.array(feats).astype('float32'), np.array(labels)

dna_train, prot_train, num_train, y_train = veri_hazirla()
print(f">> TOPLAM EĞİTİM VERİSİ: {len(y_train)} adet.")

# --- 3. TOKENIZATION ---
dna_tok = Tokenizer(char_level=True)
dna_tok.fit_on_texts(dna_train)
prot_tok = Tokenizer(char_level=True)
prot_tok.fit_on_texts(prot_train)

X_dna = pad_sequences(dna_tok.texts_to_sequences(dna_train), maxlen=11, padding='post')
X_prot = pad_sequences(prot_tok.texts_to_sequences(prot_train), maxlen=11, padding='post')

# --- 4. MODEL EĞİTİMİ (STACKING) ---
# Veriyi Base (Temel Eğitim) ve Meta (Stacking Eğitimi) olarak ikiye bölüyoruz
X_dna_base, X_dna_meta, X_prot_base, X_prot_meta, X_num_base, X_num_meta, y_base, y_meta = train_test_split(
    X_dna, X_prot, num_train, y_train, test_size=0.25, random_state=42
)

if not os.path.exists('model_stack'): os.makedirs('model_stack')

# --- MODEL 1: CNN (Derin Öğrenme - Dizilimler için) ---
print("\n>> [1/9] CNN (Deep Learning) Eğitiliyor...")
in_dna = Input(shape=(11,)); emb_dna = Embedding(len(dna_tok.word_index)+1, 12)(in_dna)
x1 = GlobalMaxPooling1D()(Conv1D(64, 3, activation='relu')(emb_dna))
in_prot = Input(shape=(11,)); emb_prot = Embedding(len(prot_tok.word_index)+1, 12)(in_prot)
x2 = GlobalMaxPooling1D()(Conv1D(64, 3, activation='relu')(emb_prot))
in_num = Input(shape=(6,)); x3 = BatchNormalization()(Dense(32, activation='relu')(in_num))
merged = Concatenate()([x1, x2, x3])
out = Dense(1, activation='sigmoid')(Dropout(0.3)(Dense(32, activation='relu')(merged)))

model_cnn = Model(inputs=[in_dna, in_prot, in_num], outputs=out)
model_cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# CNN için Callback
class BestEpoch(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 10 == 0: print(f"    - Epoch {epoch} devam ediyor...")

model_cnn.fit([X_dna_base, X_prot_base, X_num_base], y_base, epochs=30, batch_size=32, verbose=0, callbacks=[BestEpoch()])
model_cnn.save('model_stack/model_cnn.h5')

# --- DİĞER MAKİNE ÖĞRENMESİ MODELLERİ (Sayısal Veriler İçin) ---
# Bu modeller sadece sayısal (num) verileri kullanacak

print(">> [2/9] XGBoost Eğitiliyor...")
model_xgb = xgb.XGBClassifier(n_estimators=150, max_depth=5, learning_rate=0.1).fit(X_num_base, y_base)

print(">> [3/9] LightGBM Eğitiliyor...")
model_lgbm = lgb.LGBMClassifier(n_estimators=150, verbose=-1).fit(X_num_base, y_base)

print(">> [4/9] Random Forest Eğitiliyor...")
model_rf = RandomForestClassifier(n_estimators=150, random_state=42).fit(X_num_base, y_base)

print(">> [5/9] Decision Tree Eğitiliyor...")
model_dt = DecisionTreeClassifier(random_state=42).fit(X_num_base, y_base)

print(">> [6/9] Support Vector Machine (SVM) Eğitiliyor...")
# SVM probability=True olmalı ki olasılık döndürebilsin
model_svm = SVC(probability=True, random_state=42).fit(X_num_base, y_base)

print(">> [7/9] K-Nearest Neighbors (KNN) Eğitiliyor...")
model_knn = KNeighborsClassifier(n_neighbors=5).fit(X_num_base, y_base)

print(">> [8/9] Naive Bayes (Gaussian) Eğitiliyor...")
model_nb = GaussianNB().fit(X_num_base, y_base)

print(">> [9/9] Lojistik Regresyon (Base) Eğitiliyor...")
model_lr_base = LogisticRegression(max_iter=1000).fit(X_num_base, y_base)

# --- 5. META MODEL (STACKING) ---
print("\n>> [META] Tüm Modellerin Tahminleri Birleştiriliyor...")

# Meta seti üzerinde her modelin tahminini al (Olasılık olarak)
# Not: CNN tek sütun verir, diğerleri (N,2) verir o yüzden [:, 1] alıyoruz.
p_cnn = model_cnn.predict([X_dna_meta, X_prot_meta, X_num_meta], verbose=0).flatten()
p_xgb = model_xgb.predict_proba(X_num_meta)[:, 1]
p_lgbm = model_lgbm.predict_proba(X_num_meta)[:, 1]
p_rf = model_rf.predict_proba(X_num_meta)[:, 1]
p_dt = model_dt.predict_proba(X_num_meta)[:, 1]
p_svm = model_svm.predict_proba(X_num_meta)[:, 1]
p_knn = model_knn.predict_proba(X_num_meta)[:, 1]
p_nb = model_nb.predict_proba(X_num_meta)[:, 1]
p_lr = model_lr_base.predict_proba(X_num_meta)[:, 1]

# Tüm tahminleri yan yana diz (Stacking Input)
# 9 Modelin tahmini yan yana gelerek yeni bir özellik seti oluşturur
stack_X_meta = np.column_stack((p_cnn, p_xgb, p_lgbm, p_rf, p_dt, p_svm, p_knn, p_nb, p_lr))

# Meta Model (Hakem) Eğitimi
meta_model = LogisticRegression()
meta_model.fit(stack_X_meta, y_meta)

# --- 6. SONUÇLARI DEĞERLENDİRME ---
final_preds = meta_model.predict(stack_X_meta)
final_acc = accuracy_score(y_meta, final_preds)
final_f1 = f1_score(y_meta, final_preds)

print("\n" + "="*50)
print(f"🏆 FİNAL SONUÇLARI (9 MODEL BİRLEŞİMİ)")
print(f"✅ Accuracy : %{final_acc*100:.2f}")
print(f"🚀 F1 Score : %{final_f1*100:.2f}")
print("-" * 50)
print("Detaylı Rapor:")
print(classification_report(y_meta, final_preds, target_names=["Sağlıklı", "Hasta"]))
print("="*50)

# --- KAYDETME İŞLEMLERİ ---
print(">> Modeller kaydediliyor...")
joblib.dump(meta_model, 'model_stack/meta_model.pkl')
joblib.dump(model_xgb, 'model_stack/xgb.pkl')
joblib.dump(model_lgbm, 'model_stack/lgbm.pkl')
joblib.dump(model_rf, 'model_stack/rf.pkl')
joblib.dump(model_dt, 'model_stack/dt.pkl')
joblib.dump(model_svm, 'model_stack/svm.pkl')
joblib.dump(model_knn, 'model_stack/knn.pkl')
joblib.dump(model_nb, 'model_stack/nb.pkl')
joblib.dump(model_lr_base, 'model_stack/lr_base.pkl')

with open('model_stack/dna_tok.pickle', 'wb') as f: pickle.dump(dna_tok, f)
with open('model_stack/prot_tok.pickle', 'wb') as f: pickle.dump(prot_tok, f)

print("✅ HER ŞEY HAZIR! TEKNOFEST SİMÜLASYONU TAMAM.")

>> SİSTEM BAŞLATILIYOR (FULL MODEL MODU)...

🤔 EĞİTİM STRATEJİSİ
>> Kaç adet 'MUALLAK' (Arada kalmış) veri üreteyim? (Örn: 200): 5000

>> Veriler Harmanlanıyor...
   ⚠️ 5000 adet 'Muallak/Zor' veri üretiliyor...
>> TOPLAM EĞİTİM VERİSİ: 9300 adet.

>> [1/9] CNN (Deep Learning) Eğitiliyor...
    - Epoch 0 devam ediyor...
    - Epoch 10 devam ediyor...
    - Epoch 20 devam ediyor...


>> [2/9] XGBoost Eğitiliyor...
>> [3/9] LightGBM Eğitiliyor...
>> [4/9] Random Forest Eğitiliyor...
>> [5/9] Decision Tree Eğitiliyor...
>> [6/9] Support Vector Machine (SVM) Eğitiliyor...
>> [7/9] K-Nearest Neighbors (KNN) Eğitiliyor...
>> [8/9] Naive Bayes (Gaussian) Eğitiliyor...
>> [9/9] Lojistik Regresyon (Base) Eğitiliyor...

>> [META] Tüm Modellerin Tahminleri Birleştiriliyor...


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



🏆 FİNAL SONUÇLARI (9 MODEL BİRLEŞİMİ)
✅ Accuracy : %99.53
🚀 F1 Score : %99.51
--------------------------------------------------
Detaylı Rapor:
              precision    recall  f1-score   support

    Sağlıklı       0.99      1.00      1.00      1194
       Hasta       1.00      0.99      1.00      1131

    accuracy                           1.00      2325
   macro avg       1.00      1.00      1.00      2325
weighted avg       1.00      1.00      1.00      2325

>> Modeller kaydediliyor...
✅ HER ŞEY HAZIR! TEKNOFEST SİMÜLASYONU TAMAM.


In [10]:
# DOSYA ADI: canli_test_zorlu.py
import numpy as np
import pickle
import joblib
import time
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

# Renkler (Terminalde güzel görünmesi için)
YESIL = "\033[92m"
KIRMIZI = "\033[91m"
MAVI = "\033[94m"
SARI = "\033[93m"
CYAN = "\033[96m"
RESET = "\033[0m"

# Uyarıları gizle
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print(f"{SARI}>> CANLI TEST BAŞLIYOR (9 Model + Detaylı Metrik Raporu)...{RESET}")

# --- 1. MODELLERİ YÜKLE ---
print(">> Modeller yükleniyor, lütfen bekleyin...")
try:
    # Tokenizer'lar
    with open('model_stack/dna_tok.pickle', 'rb') as f: dna_tok = pickle.load(f)
    with open('model_stack/prot_tok.pickle', 'rb') as f: prot_tok = pickle.load(f)

    # 1. Derin Öğrenme
    model_cnn = load_model('model_stack/model_cnn.h5')

    # 2. Klasik Makine Öğrenmesi Modelleri
    model_xgb = joblib.load('model_stack/xgb.pkl')
    model_lgbm = joblib.load('model_stack/lgbm.pkl')
    model_rf = joblib.load('model_stack/rf.pkl')
    model_dt = joblib.load('model_stack/dt.pkl')
    model_svm = joblib.load('model_stack/svm.pkl')
    model_knn = joblib.load('model_stack/knn.pkl')
    model_nb = joblib.load('model_stack/nb.pkl')
    model_lr_base = joblib.load('model_stack/lr_base.pkl')

    # 3. META MODEL (Karar Verici)
    meta_model = joblib.load('model_stack/meta_model.pkl')

    print(f"{YESIL}>> Tüm modeller başarıyla yüklendi!{RESET}")

except FileNotFoundError as e:
    print(f"{KIRMIZI}❌ HATA: Dosya bulunamadı! Önce eğitim kodunu çalıştırın.{RESET}")
    exit()
except Exception as e:
    print(f"{KIRMIZI}❌ Beklenmeyen bir hata: {e}{RESET}")
    exit()

# --- 2. YARDIMCI FONKSİYONLAR ---
def generate_sequence_with_pattern(length, vocab, pattern=None):
    seq = list(np.random.choice(list(vocab), size=length))
    if pattern:
        start = np.random.randint(0, length - len(pattern) + 1)
        for i, char in enumerate(pattern):
            seq[start + i] = char
    return "".join(seq)

# --- 3. TEST VERİSİ ÜRETİCİ ---
def canli_veri_uret():
    risk = np.random.uniform(0.1, 0.9)
    cons = np.random.uniform(0, 10)
    maf = np.random.exponential(0.05)
    hydro = np.random.uniform(-5, 5)
    polar = np.random.uniform(-3, 3)
    weight_base = np.random.uniform(-50, 50)

    # Formül (Ground Truth)
    score = (risk * 0.50) + (cons/10 * 0.30) + ((0.5 - maf) * 0.1) + (abs(hydro)/5 * 0.1)

    muallak_mi = 0.55 < score < 0.65
    gercek_etiket = 1 if score > 0.60 else 0

    # Motif Ekleme (Sinyal)
    if gercek_etiket == 1:
        dna = generate_sequence_with_pattern(11, "ACGT", pattern="CGT")
        prot = generate_sequence_with_pattern(11, "ACDEFGHIKLMNPQRSTVWY", pattern="WW")
        weight = weight_base + 10
    else:
        dna = generate_sequence_with_pattern(11, "ACGT", pattern="AAA")
        prot = generate_sequence_with_pattern(11, "ACDEFGHIKLMNPQRSTVWY", pattern="LL")
        weight = weight_base - 10

    return dna, prot, [risk, maf, cons, hydro, polar, weight], gercek_etiket, muallak_mi

# --- 4. TAHMİN MEKANİZMASI ---
def tahmin_et(dna, prot, feats):
    s_d = pad_sequences(dna_tok.texts_to_sequences([dna]), maxlen=11, padding='post')
    s_p = pad_sequences(prot_tok.texts_to_sequences([prot]), maxlen=11, padding='post')
    f_n = np.array([feats]).astype('float32')

    # Modellerden tahmin al
    p_cnn = model_cnn.predict([s_d, s_p, f_n], verbose=0).flatten()[0]
    p_xgb = model_xgb.predict_proba(f_n)[:, 1][0]
    p_lgbm = model_lgbm.predict_proba(f_n)[:, 1][0]
    p_rf = model_rf.predict_proba(f_n)[:, 1][0]
    p_dt = model_dt.predict_proba(f_n)[:, 1][0]
    p_svm = model_svm.predict_proba(f_n)[:, 1][0]
    p_knn = model_knn.predict_proba(f_n)[:, 1][0]
    p_nb = model_nb.predict_proba(f_n)[:, 1][0]
    p_lr = model_lr_base.predict_proba(f_n)[:, 1][0]

    # Stacking
    stack_input = np.array([[p_cnn, p_xgb, p_lgbm, p_rf, p_dt, p_svm, p_knn, p_nb, p_lr]])

    final_pred = meta_model.predict(stack_input)[0]
    final_prob = meta_model.predict_proba(stack_input)[:, 1][0]

    return final_pred, final_prob, p_cnn

# --- 5. SİMÜLASYON VE METRİK HESAPLAMA ---
y_true = [] # Gerçek değerleri tutacak liste
y_pred = [] # Tahminleri tutacak liste

TEST_ADEDI = 300 # İstatistiksel anlam için en az 30 önerilir

print(f"\n{'TÜR':<10} | {'DNA (Motif)':<12} | {'RİSK':<5} | {'CNN GÜVEN':<10} | {'GERÇEK':<8} | {'TAHMİN':<8} | {'SONUÇ'}")
print("-" * 95)

for i in range(TEST_ADEDI):
    dna, prot, feats, gercek, muallak = canli_veri_uret()
    pred, prob, cnn_conf = tahmin_et(dna, prot, feats)

    # Listelere kaydet (Metrik hesabı için)
    y_true.append(gercek)
    y_pred.append(pred)

    str_gercek = "HASTA" if gercek == 1 else "SAĞLAM"
    str_tahmin = "HASTA" if pred == 1 else "SAĞLAM"

    if gercek == pred:
        ikon = f"{YESIL}✅{RESET}"
    else:
        ikon = f"{KIRMIZI}❌{RESET}"

    tur_str = f"{MAVI}[MUALLAK]{RESET}" if muallak else " [NET]   "
    dna_goster = dna[:8] + "..."

    print(f"{tur_str:<10} | {dna_goster:<12} | {feats[0]:.2f}  | %{cnn_conf*100:<9.1f} | {str_gercek:<8} | {str_tahmin:<8} | {ikon}")
    time.sleep(0.1)

# --- 6. DETAYLI METRİK RAPORU ---
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)
kappa = cohen_kappa_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)

print("\n" + "="*50)
print(f"📊 PERFORMANS KARNESİ ({TEST_ADEDI} Test Üzerinden)")
print("="*50)

# 1. TEMEL METRİKLER
print(f"✅ Doğruluk (Accuracy)  : {CYAN}%{acc*100:.2f}{RESET}")
print(f"🎯 Kesinlik (Precision) : {CYAN}%{prec*100:.2f}{RESET}  (Hasta dediklerimin ne kadarı gerçekten hasta?)")
print(f"🔎 Duyarlılık (Recall)  : {CYAN}%{rec*100:.2f}{RESET}  (Gerçek hastaların ne kadarını yakaladım?)")
print(f"🚀 F1 Skoru             : {CYAN}%{f1*100:.2f}{RESET}  (Precision ve Recall dengesi - EN ÖNEMLİSİ)")
print(f"🧠 Cohen's Kappa        : {CYAN}{kappa:.4f}{RESET}   (Şans faktöründen arındırılmış başarı)")

# 2. CONFUSION MATRIX (KARMAŞIKLIK MATRİSİ) GÖRSELİ
try:
    tn, fp, fn, tp = cm.ravel()
except ValueError:
    # Eğer test çok kısaysa ve sadece tek sınıf varsa hata verebilir, onu yönetelim
    tn, fp, fn, tp = 0, 0, 0, 0

print("\n🧩 Karmaşıklık Matrisi (Detay):")
print(f"┌───────────────────────┬───────────────────────┐")
print(f"│     SAĞLAM (Tahmin)   │     HASTA (Tahmin)    │")
print(f"├───────────────────────┼───────────────────────┤")
print(f"│ Gerçek SAĞLAM: {YESIL}{tn:<5}{RESET}  │ Yanlış Alarm:  {KIRMIZI}{fp:<5}{RESET}  │")
print(f"│ Kaçırılan:     {KIRMIZI}{fn:<5}{RESET}  │ Gerçek HASTA:  {YESIL}{tp:<5}{RESET}  │")
print(f"└───────────────────────┴───────────────────────┘")

# 3. YORUM
print("-" * 50)
if f1 > 0.90:
    print(f"{YESIL}🌟 SONUÇ: MÜKEMMEL! Model yarışmaya hazır.{RESET}")
elif f1 > 0.75:
    print(f"{SARI}⚠️ SONUÇ: İYİ. Ancak 'Muallak' verilerde biraz zorlanıyor olabilir.{RESET}")
else:
    print(f"{KIRMIZI}❌ SONUÇ: GELİŞTİRİLMELİ. Veri setini artırmayı dene.{RESET}")
print("="*50)

>> CANLI TEST BAŞLIYOR (9 Model + Detaylı Metrik Raporu)...
>> Modeller yükleniyor, lütfen bekleyin...
>> Tüm modeller başarıyla yüklendi!

TÜR        | DNA (Motif)  | RİSK  | CNN GÜVEN  | GERÇEK   | TAHMİN   | SONUÇ
-----------------------------------------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[MUALLAK] | AAAACTAA...  | 0.56  | %43.7      | SAĞLAM   | HASTA    | ❌


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[MUALLAK] | CGTATCGC...  | 0.50  | %100.0     | HASTA    | HASTA    | ✅


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 [NET]     | AAACTCGA...  | 0.47  | %0.0       | SAĞLAM   | SAĞLAM   | ✅


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


 [NET]     | AAAAGTGC...  | 0.11  | %0.0       | SAĞLAM   | SAĞLAM   | ✅
[MUALLAK] | CGTACTAG...  | 0.69  | %100.0     | HASTA    | HASTA    | ✅

📊 PERFORMANS KARNESİ (300 Test Üzerinden)
✅ Doğruluk (Accuracy)  : %77.00
🎯 Kesinlik (Precision) : %53.57  (Hasta dediklerimin ne kadarı gerçekten hasta?)
🔎 Duyarlılık (Recall)  : %94.94  (Gerçek hastaların ne kadarını yakaladım?)
🚀 F1 Skoru             : %68.49  (Precision ve Recall dengesi - EN ÖNEMLİSİ)
🧠 Cohen's Kappa        : 0.5250   (Şans faktöründen arındırılmış başarı)

🧩 Karmaşıklık Matrisi (Detay):
┌───────────────────────┬───────────────────────┐
│     SAĞLAM (Tahmin)   │     HASTA (Tahmin)    │
├───────────────────────┼───────────────────────┤
│ Gerçek SAĞLAM: 156    │ Yanlış Alarm:  65     │
│ Kaçırılan:     4      │ Gerçek HASTA:  75     │
└───────────────────────┴───────────────────────┘
--------------------------------------------------
❌ SONUÇ: GELİŞTİRİLMELİ. Veri setini artırmayı dene.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
